# Preprocessing Cleaned TDS Data for NLP Analysis 
### *Claps as Target Variable*

In [1]:
import pandas as pd
import numpy as np


#nlp
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction import text
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rachelinsler/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
filename = 'data/cleaned_no_outliers.csv'

In [4]:
df = pd.read_csv(filename)

In [5]:
df.head()

,date,title,subtitle,claps,responses,author_url,story_url,reading_time (mins),number_sections,section_titles,number_paragraphs,paragraphs
0,01/01/2021,The Ultimate Guide to Acing Coding Interviews ...,Data Science Interview,489,4,https://towardsdatascience.com/@emmading,https://towardsdatascience.com/the-ultimate-gu...,11,12,['The Ultimate Guide to Acing Coding Interview...,42,"['Emma Ding', 'Jan 1·11 min read', 'Written by..."
1,01/01/2021,Shakespeare versus Eminem— who’s the better ly...,"He is known for his poetry, his writings on life…",139,2,https://towardsdatascience.com/@jeroenvanzeeland,https://towardsdatascience.com/shakespeare-ver...,9,13,['Shakespeare versus Eminem—who’s the better l...,64,"['Jeroen van Zeeland', 'Jan 1·9 min read', 'Da..."
2,01/01/2021,Implementing VisualTtransformer in PyTorch,"Hi guys, happy new year! Today we are going to...",133,2,https://towardsdatascience.com/@FrancescoZ,https://towardsdatascience.com/implementing-vi...,6,6,['Implementing Vision Transformer (ViT) in PyT...,60,"['Francesco Zuppichini', 'Jan 1·6 min read', '..."
3,01/01/2021,Stock Price Analysis with Pandas and Altair,Practical guide for Pandas and Altair,92,0 responses,https://towardsdatascience.com/@sonery,https://towardsdatascience.com/stock-price-ana...,5,1,['Stock Price Analysis with Pandas and Altair'],29,"['Soner Yıldırım', 'Jan 1·5 min read', 'Stock ..."
4,01/01/2021,Optimal Threshold for Imbalanced Classification,Hands-on Tutorial,58,1,https://towardsdatascience.com/@audhiaprilliant,https://towardsdatascience.com/optimal-thresho...,9,1,['Optimal Threshold for Imbalanced Classificat...,40,"['Audhi Aprilliant', 'Jan 1·9 min read', 'Clas..."


In [6]:
df.columns

Index(['date', 'title', 'subtitle', 'claps', 'responses', 'author_url',
       'story_url', 'reading_time (mins)', 'number_sections', 'section_titles',
       'number_paragraphs', 'paragraphs'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9290 entries, 0 to 9289
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 9290 non-null   object
 1   title                9290 non-null   object
 2   subtitle             9290 non-null   object
 3   claps                9290 non-null   int64 
 4   responses            9290 non-null   object
 5   author_url           9290 non-null   object
 6   story_url            9290 non-null   object
 7   reading_time (mins)  9290 non-null   int64 
 8   number_sections      9290 non-null   int64 
 9   section_titles       9290 non-null   object
 10  number_paragraphs    9290 non-null   int64 
 11  paragraphs           9290 non-null   object
dtypes: int64(4), object(8)
memory usage: 871.1+ KB


In [8]:
#remove non-text, non-Y columns
df = df.drop(columns=['date', 'responses', 'author_url',
       'story_url', 'reading_time (mins)', 'number_sections', 
       'number_paragraphs'])

#### Combine Title & Subtitle

In [9]:
df['combined_title'] = (df['title'] + ' ' + df['subtitle'])

In [10]:
df = df.drop(columns = ['title', 'subtitle']).reset_index(drop=True)

#### Remove Punctuation & Make Lowercase

In [11]:
text_cols = ['section_titles', 'paragraphs', 'combined_title']

In [12]:
for col in text_cols:
    df[col] = df[col].map(lambda x: re.sub("[''·,\.!?'-]", '', x).lower())

#### Remove the 'Min Read' Phrase at the Beginning of Each Paragraph

In [13]:
#remove all text up to and including 'min read' in the beginning of the paragraphs
df['paragraphs'] = df['paragraphs'].map(lambda x: (x[x.index('min read')+9:] if (x.count('min read') > 0) else x))
                     

#### Remove the Brackets Surrounding The Section Titles

In [14]:
#remove brackets surrounding the section titles
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("[",""))
df['section_titles'] = df['section_titles'].map(lambda x: x.replace("]",""))

In [15]:
df.head()

,claps,section_titles,paragraphs,combined_title
0,489,the ultimate guide to acing coding interviews ...,written by emma ding and rob wang data science...,the ultimate guide to acing coding interviews ...
1,139,shakespeare versus eminem—who’s the better lyr...,data science has crept into every conceivable ...,shakespeare versus eminem— who’s the better ly...
2,133,implementing vision transformer (vit) in pytor...,hi guys happy new year today we are going to i...,implementing visualttransformer in pytorch hi ...
3,92,stock price analysis with pandas and altair,stock price analysis is an example of time ser...,stock price analysis with pandas and altair pr...
4,58,optimal threshold for imbalanced classification,classification is one of the supervised learni...,optimal threshold for imbalanced classificatio...


#### Tokenize Text

In [16]:
w_tokenizer = WhitespaceTokenizer()

In [17]:
for col in text_cols:
    df[col] = df[col].apply(lambda x: w_tokenizer.tokenize(x))

In [18]:
df.head()

,claps,section_titles,paragraphs,combined_title
0,489,"[the, ultimate, guide, to, acing, coding, inte...","[written, by, emma, ding, and, rob, wang, data...","[the, ultimate, guide, to, acing, coding, inte..."
1,139,"[shakespeare, versus, eminem—who’s, the, bette...","[data, science, has, crept, into, every, conce...","[shakespeare, versus, eminem—, who’s, the, bet..."
2,133,"[implementing, vision, transformer, (vit), in,...","[hi, guys, happy, new, year, today, we, are, g...","[implementing, visualttransformer, in, pytorch..."
3,92,"[stock, price, analysis, with, pandas, and, al...","[stock, price, analysis, is, an, example, of, ...","[stock, price, analysis, with, pandas, and, al..."
4,58,"[optimal, threshold, for, imbalanced, classifi...","[classification, is, one, of, the, supervised,...","[optimal, threshold, for, imbalanced, classifi..."


#### Remove Special Characters 

In [19]:
for col in text_cols:
    df[col] = df[col].replace('[^\w]','',regex=True)

In [20]:
df.head()

,claps,section_titles,paragraphs,combined_title
0,489,"[the, ultimate, guide, to, acing, coding, inte...","[written, by, emma, ding, and, rob, wang, data...","[the, ultimate, guide, to, acing, coding, inte..."
1,139,"[shakespeare, versus, eminem—who’s, the, bette...","[data, science, has, crept, into, every, conce...","[shakespeare, versus, eminem—, who’s, the, bet..."
2,133,"[implementing, vision, transformer, (vit), in,...","[hi, guys, happy, new, year, today, we, are, g...","[implementing, visualttransformer, in, pytorch..."
3,92,"[stock, price, analysis, with, pandas, and, al...","[stock, price, analysis, is, an, example, of, ...","[stock, price, analysis, with, pandas, and, al..."
4,58,"[optimal, threshold, for, imbalanced, classifi...","[classification, is, one, of, the, supervised,...","[optimal, threshold, for, imbalanced, classifi..."


#### Remove English Stopwords

In [21]:
stop_words = set(stopwords.words('english')) 

In [22]:
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

In [23]:
for col in text_cols:
    df[col] = df[col].apply(remove_stopwords)

#### Lemmatize

Uncomment this section to save the file with lemmatized text.

In [24]:
# def lemmatize_text(text):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(w) for w in text]

In [25]:
# for col in text_cols:
#     df[col] = df[col].apply(lemmatize_text)

In [26]:
# save as lemmatized
# output_filename = 'data/nlp_nltk_lemmatized_preproc.csv'

#### Porter Stemmer
Uncomment this section to save the file with stemmed text.

In [27]:
# def stem_text(text):
#     stemmer = PorterStemmer()
#     return [stemmer.stem(w) for w in text]

In [28]:
# for col in text_cols:
#     df[col] = df[col].apply(stem_text)

In [29]:
# output_filename = 'data/nlp_nltk_stemmed_preproc.csv'

#### No Stemming
Uncomment this section to save the file without any lemmatization or stemmed. 

In [30]:
#output filename for no lemmatizing or stemming
output_filename = 'data/nlp_nltk_nostem_preproc_strings.csv'

#### Combine Text Columns

In [31]:
df['text'] = df['combined_title'] + df['section_titles'] + df['paragraphs']

In [32]:
tds = df.drop(columns = ['section_titles', 'paragraphs', 'combined_title'])

In [33]:
tds['text'] = [' '.join(x) for x in tds['text']]

In [34]:
tds.head()

,claps,text
0,489,ultimate guide acing coding interviews data sc...
1,139,shakespeare versus eminem— who’s better lyrici...
2,133,implementing visualttransformer pytorch hi guy...
3,92,stock price analysis pandas altair practical g...
4,58,optimal threshold imbalanced classification ha...


#### Save Pre-Processed File

In [35]:
tds.to_csv(output_filename)